In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import itertools
from datetime import datetime, timedelta, date
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from arch.unitroot import engle_granger
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
from bs4 import BeautifulSoup
import requests, requests_html
warnings.simplefilter('ignore')
from numpy import log, polyfit, sqrt, std, subtract
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from pykalman import KalmanFilter
from datetime import datetime
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import itertools
from datetime import datetime, timedelta, date
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from arch.unitroot import engle_granger
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
from bs4 import BeautifulSoup
import requests, requests_html
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import itertools
from datetime import datetime, timedelta, date
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from arch.unitroot import engle_granger
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
from bs4 import BeautifulSoup
import requests, requests_html
warnings.simplefilter('ignore')

In [ ]:
import yfinance as yf
cs = pd.read_excel('tickers.xlsx')
symbList = cs['Tickers'].values.tolist()
df = yf.download(symbList,'2016-9-1','2020-9-1')['Adj Close']
print(df.head())
df = df.dropna(axis=1, how='all')
df.head()

In [ ]:
df = df[1:]
limitPer = len(df) * .50
df = df.dropna(thresh=limitPer,axis=1)
df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(method='ffill')

In [ ]:
split_train = .7
train_sample = int(split_train * len(df))
# creates combinations of all tickers within the selected index
symbol_pairs = list(itertools.combinations(symbList, 2))
significance_level = 0.05

# selects pairs based on Engle-Granger cointegration test 
def find_cointegrated_pairs(dataframe):
    coint_pairs = []
    for y, x in symbol_pairs:
            try:
                eg_test = engle_granger(dataframe[y], dataframe[x], trend="n")
                print(f'{y} - {x} : p-value = {eg_test.pvalue}')
                if eg_test.pvalue < significance_level:
                    coint_pairs.append((y, x, eg_test.pvalue))
            except:
                print(f'Oops! Nans detected. The pair: {x} - {y} will be skipped.')
    return coint_pairs

find_cointegrated_pairs(df[0:train_sample])

In [ ]:
pvalue_list_sorted = coint_pairs
pvalue_list_sorted = sorted(coint_pairs, key=lambda x: abs(x[2]),reverse = False) 
top_pairs = 1000
top_coint_pairs_list = []

for c, pair in enumerate(pvalue_list_sorted, 1):
    if len(top_coint_pairs_list) < top_pairs:
        
        
        
        top_coint_pairs_list.append((pair[0], pair[1], pair[2]))
        print(f'{c}: |{pair[0]: <4}| - |{pair[1]: <4}| p-value: {round(pair[2],3)}')
    else:
        break

In [ ]:
def hurst(ts):
    """Returns the Hurst Exponent of the time series vector ts"""
    # Create the range of lag values
    lags = range(2, 100)
 
    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
 
    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)
 
    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0

###############################################################################
##### ADF TEST
###############################################################################
def adf_test(x, y):
    df = pd.DataFrame({'y':y,'x':x})
    est = sm.OLS(df.y, df.x)
    est = est.fit()
    df['hr'] = -est.params[0]
    df['spread'] = df.y + (df.x * df.hr)
    
    cadf = ts.adfuller(df.spread)   
    return cadf[1] 
def half_life(spread):
    spread_lag = spread.shift(1)
    spread_lag.iloc[0] = spread_lag.iloc[1]
    
    spread_ret = spread - spread_lag
    spread_ret.iloc[0] = spread_ret.iloc[1]
    
    spread_lag2 = sm.add_constant(spread_lag)
     
    model = sm.OLS(spread_ret,spread_lag2)
    res = model.fit()
    halflife = int(round(-np.log(2) / res.params[1],0))
 
    if halflife <= 0:
        halflife = 1
    return halflife
    
def KalmanFilterAverage(x):
    # Construct a Kalman filter
    from pykalman import KalmanFilter
    kf = KalmanFilter(transition_matrices = [1],
                      observation_matrices = [1],
                      initial_state_mean = 0,
                      initial_state_covariance = 1,
                      observation_covariance=1,
                      transition_covariance=.01)
# Use the observed values of the price to get a rolling mean
    state_means, _ = kf.filter(x.values)
    state_means = pd.Series(state_means.flatten(), index=x.index)
    return state_means
## Kalman filter regression
def KalmanFilterRegression(x,y):
    delta = 1e-3
    trans_cov = delta / (1 - delta) * np.eye(2) # How much random walk wiggles
    obs_mat = np.expand_dims(np.vstack([[x], [np.ones(len(x))]]).T, axis=1)
    
    kf = KalmanFilter(n_dim_obs=1, n_dim_state=2, # y is 1-dimensional, (alpha, beta) is 2-dimensional
                      initial_state_mean=[0,0],
                      initial_state_covariance=np.ones((2, 2)),
                      transition_matrices=np.eye(2),
                      observation_matrices=obs_mat,
                      observation_covariance=2,
                      transition_covariance=trans_cov)
    
    # Use the observations y to get running estimates and errors for the state parameters
    state_means, state_covs = kf.filter(y.values)
    return state_means

In [ ]:
def backtest(df,sym1,sym2,strategy,spread_calculation,entryZscore,exitZscore,stoploss_Zscore):

    
    
#     """BACKTESTING SELECTED PAIR.....

#     Parameters
#     ----------
#     dataframe : pandas dataframe
#         All assets close prices
#     sym1 : string
#         The symbol of asset one
#     sym2 : string
#         The symbol of asset two
#     strategy : int
#         Trading strategy selection {1,2}.
#     roll_beta_window : integer 
#     zscore_window : integer
#     entryZscore : float
#     exitZscore : float

#     Returns
#     -------
#     dataframe[f'Cum_rets: {sym1} {sym2}'], CAGR, sharpe, num_days_in_market : pd.series
#         Time series of cumulative returns for selected pair
#     comp_ann_return_net, annualised_sharpe_net , num_days_in_the_market : float
#         Compound annual growth rate (CAGR) for selected pair
#    annualised_sharpe_net , num_days_in_the_market : float
#         Sharpe Ratio for selected pair
#    num_days_in_the_market : integer
#         Number of days the pair is 'in the market'

#     """
    
    y = df[sym1]
    x = df[sym2]
    intersect = y.index.intersection(x.index)
    y = y.loc[intersect]
    x = x.loc[intersect]

    # creates a dataframe using Adj. Close prices from both series
    dataframe = pd.DataFrame(index=y.index).dropna()
    
    dataframe['%s_close' % sym1.lower()] = y
    dataframe['%s_close' % sym2.lower()] = x  
    if spread_calculation == 1:
        state_means = KalmanFilterRegression(KalmanFilterAverage(x),KalmanFilterAverage(y))
        dataframe['hr'] = - state_means[:,0]
        dataframe['spread'] = y + (x * dataframe.hr)
        halflife = half_life(dataframe['spread'])
    if spread_calculation == 2:
        est = sm.OLS(y,x)
        est = est.fit()
        dataframe['hr'] = -est.params[0]

        dataframe['spread'] = y + (x * dataframe.hr)
        halflife = half_life(dataframe['spread'])
    

    ##############################################################
    

    ##########################################################

    meanSpread = dataframe.spread.rolling(window=halflife).mean()
    stdSpread = dataframe.spread.rolling(window=halflife).std()
    
        
    dataframe['zScore'] = (dataframe.spread-meanSpread)/stdSpread
   

    # selects a trading model : 1 or 2
    #     :1 (standard model): buy/sell when z-score reach an entry threshold, 
    #        and exit when z-score reaches exit threshold.
    #     :2 (alternative model): buy/sell when z-score reach an entry threshold, 
    #        and the current z-score level is lower/higher that the previous one.

    if strategy == 1:
    # code below (only for model == 1) borrowed from (https://www.pythonforfinance.net/)
        dataframe['long_entry'] = ((dataframe.zScore < - entryZscore) & ( dataframe.zScore.shift(1) > - entryZscore)) 
        dataframe['long_exit'] = ((dataframe.zScore > - exitZscore) & (dataframe.zScore.shift(1) < - exitZscore)) 
        dataframe['stop_loss_exit_long'] = ((dataframe.zScore < - stoploss_Zscore) & ( dataframe.zScore.shift(1) > - stoploss_Zscore))
        dataframe.loc[dataframe['long_entry'],'pos_long'] = 1 
        dataframe.loc[dataframe['long_exit'],'pos_long'] = 0 
        dataframe.loc[dataframe['stop_loss_exit_long'],'pos_long'] = 0
        dataframe['pos_long'][0] = 0 
        dataframe['pos_long'] = dataframe['pos_long'].fillna(method='pad') 

        #calculate when portfolio is SHORT
        dataframe['short_entry'] = ((dataframe.zScore > entryZscore) & ( dataframe.zScore.shift(1) < entryZscore))
        dataframe['short_exit'] = ((dataframe.zScore < exitZscore) & (dataframe.zScore.shift(1) > exitZscore))
        dataframe['stop_loss_exit_short'] = ((dataframe.zScore <  stoploss_Zscore) & ( dataframe.zScore.shift(1) >  stoploss_Zscore))
        dataframe.loc[dataframe['short_entry'],'pos_short'] = -1
        dataframe.loc[dataframe['short_exit'],'pos_short'] = 0
        dataframe.loc[dataframe['stop_loss_exit_short'],'pos_short'] = 0
        dataframe['pos_short'][0] = 0
        dataframe['pos_short'] = dataframe['pos_short'].fillna(method='pad')
    
    if strategy == 2:
        dataframe['long_entry'] = ((dataframe.zScore < -entryZscore) & ( dataframe.zScore < dataframe.zScore.shift(1))) 
        dataframe.loc[dataframe['long_entry'],'pos_long'] = 1 
        dataframe['pos_long'] = dataframe['pos_long'].fillna(0) 
        dataframe['long_exit'] = ((dataframe['pos_long'] == 0) & ( dataframe['pos_long'].shift(1) == 1))
        dataframe['short_entry'] = ((dataframe.zScore > entryZscore) & ( dataframe.zScore > dataframe.zScore.shift(1)))
        dataframe.loc[dataframe['short_entry'],'pos_short'] = -1
        dataframe['pos_short'] = dataframe['pos_short'].fillna(0)
        dataframe['short_exit'] = ((dataframe['pos_short'] == 0) & ( dataframe['pos_short'].shift(1) == -1)) 
        
    # combine longs/shorts and remove Look ahead bias by lagging the signal
    dataframe['position'] = dataframe['pos_long'].shift(1) + dataframe['pos_short'].shift(1)

    #########################################
    # Override entry/exit columns with entry/exit data 
    dataframe['long_entry'] = ((dataframe.pos_long.shift(1) == 1) & ((dataframe.position - dataframe.position.shift(1)) == 1)) * 1
    dataframe['long_exit'] = ((dataframe.long_exit == True) & (dataframe.position == 1)) * 1
    dataframe['stop_loss_long_exit'] = ((dataframe.stop_loss_exit_long == True) & (dataframe.position == 1)) * 1
    dataframe['short_entry'] = ((dataframe.pos_short.shift(1) == -1)  & ((dataframe.position - dataframe.position.shift(1)) == -1)) * 1
    dataframe['short_exit'] = ((dataframe.short_exit == True) & (dataframe.position == -1)) * 1
    dataframe['stop_loss_short_exit'] = ((dataframe.stop_loss_exit_short == True) & (dataframe.position == -1)) * 1
    

    # calculates adjusted spread using using current prices and hedge ratio from previous bar (avoiding look-ahead bias)
    dataframe['test_spread_adj'] = dataframe.iloc[:, 0] + (dataframe['hr'].shift(1) * dataframe.iloc[:, 1])
    
    dataframe['pct_ret'] = ((dataframe['test_spread_adj'] - dataframe['spread'].shift(1)) / 
                        (dataframe.iloc[:, 0].shift(1) + (abs(dataframe['hr'].shift(1)) * dataframe.iloc[:, 1].shift(1))))

    # calculates actual return for a day according to your position
    dataframe['port_ret'] = dataframe['position'] * dataframe['pct_ret'] 
    dataframe['port_ret'].fillna(0.0, inplace=True)

    # trading fees (set here as 0.025%:  slippage + transaction fees, for example you pay 0.25 USD per 1,000 value of trade)
    tr_costs = 0.00025
    dataframe['tr_cost_paid'] = (dataframe.long_entry | dataframe.long_exit | dataframe.short_entry | dataframe.short_exit | dataframe.stop_loss_exit_short | dataframe.stop_loss_exit_long)
    dataframe['port_ret_net'] = dataframe['port_ret'] - ( tr_costs * dataframe['tr_cost_paid'])

    # cumulative portfolio return gross and net
    dataframe['cum_port_ret_net'] = (dataframe['port_ret_net'] + 1.0).cumprod()
    
    # calculates Sharpe Ratio
    try:
        annualised_sharpe_net = np.sqrt(252) * dataframe['port_ret_net'].mean() / dataframe['port_ret_net'].std()
    except ZeroDivisionError:
        annualised_sharpe_net = 0.0
    # calculates CAGR   
    end = dataframe.cum_port_ret_net.iloc[-1]
    start = dataframe.cum_port_ret_net.iloc[0]
    days = len(dataframe.cum_port_ret_net)
    comp_ann_return_net = ((end / start) ** (252/days))  - 1.0  
    
    # calculates number of days the pair is 'in the market'
    num_days_in_the_market = len(dataframe.query("position == 1 or position  == -1")['port_ret'])
    
    # calculate sum of long and short entries
    num_trades_long = dataframe.query('long_entry == True')['long_entry'].sum()
    num_trades_short = dataframe.query('short_entry == True')['short_entry'].sum()
    stoploss_long = dataframe.query('stop_loss_long_exit == True')['stop_loss_long_exit'].sum()
    stoploss_short = dataframe.query('stop_loss_short_exit == True')['stop_loss_short_exit'].sum()
    
    # calculates total trading costs paid
    approx_tr_costs = (num_trades_long + num_trades_short) * tr_costs
    #pd.set_option('display.max_rows', 500)
    
    dataframe[f'Cum_rets: {sym1} {sym2}'] = dataframe['cum_port_ret_net']
    dataframe[f'Zscore: {sym1} {sym2}'] = dataframe['zScore']
    
    return dataframe[f'Cum_rets: {sym1} {sym2}'], comp_ann_return_net, annualised_sharpe_net , num_days_in_the_market

In [ ]:
# PERFORMS BACKTEST for the whole portfolio of pairs
threshold1 = 0.15
threshold2 = 1
all_cum_returns = []
print(f'Bactest for period: {df[train_sample:].index[0]} - {df[train_sample:].index[-1]}')
print(f'Net performance statistics (after transaction costs) for total number of pairs: {len(top_coint_pairs_list)}' )

for pair in top_coint_pairs_list:
    cum_returns, CAGR, sharpe, num_days_in_market = backtest(df[train_sample:], pair[0],pair[1], strategy = 1, spread_calculation = 2, entryZscore = 2.0, exitZscore = 0,stoploss_Zscore = 4)
    if CAGR > threshold1:
        if sharpe > threshold2:
            all_cum_returns.append(cum_returns)
            print(f' The pair {pair[0]}-{pair[1]}-> CAGR: {CAGR}, Sharpe: {sharpe}, Number of days in trade: {num_days_in_market}')
    
    #prints equity curve of each pair (for too many pairs the legend does not fit to the image)
            cum_returns.plot(figsize=(22,14),legend=True)
            

In [ ]:
 PERFORMS BACKTEST for the whole portfolio of pairs
threshold1 = 0.15
threshold2 = 1
all_cum_returns = []
print(f'Bactest for period: {df[train_sample:].index[0]} - {df[train_sample:].index[-1]}')
print(f'Net performance statistics (after transaction costs) for total number of pairs: {len(top_coint_pairs_list)}' )

for pair in top_coint_pairs_list:
    cum_returns, CAGR, sharpe, num_days_in_market = backtest(df[train_sample:], pair[0],pair[1], strategy = 1, spread_calculation = 2, entryZscore = 2.0, exitZscore = 0,stoploss_Zscore = 4)
    if CAGR > threshold1:
        if sharpe > threshold2:
            all_cum_returns.append(cum_returns)
            print(f' The pair {pair[0]}-{pair[1]}-> CAGR: {CAGR}, Sharpe: {sharpe}, Number of days in trade: {num_days_in_market}')
    
    #prints equity curve of each pair (for too many pairs the legend does not fit to the image)
            cum_returns.plot(figsize=(22,14),legend=True)

In [ ]:
# PLOTS PORTFOLIO EQUITY CURVE
total_returns = 0
for equity_curve in all_cum_returns:
    total_returns += equity_curve 
total_returns = total_returns/len(all_cum_returns)
total_returns.plot(style = 'b--', figsize=(18,12))

total_returns_pct_chg = total_returns.pct_change()
total_returns_pct_chg

try:
    portfolio_sharpe_net = np.sqrt(252) * total_returns_pct_chg.mean() / total_returns_pct_chg.std()
except ZeroDivisionError:
    portfolio_sharpe_net = 0.0

end = total_returns.iloc[-1]
start = total_returns.iloc[0]
days = len(total_returns)
portfolio_cagr_net = ((end / start) ** (252/days))  - 1.0  

print(f'Portfolio of pairs performance:')
print(f'CAGR: {round(portfolio_cagr_net,4)}, Sharpe Ratio: {round(portfolio_sharpe_net,2)}')

In [ ]:
import quantstats as qs
threshold_date = '2019-09-01'
qs.reports.full(total_returns[threshold_date:])